In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from dscribe.descriptors import ACSF
from descriptors.sf.get_sf_param import get_sf_radial_params, get_sf_ang_params

import glob
from ml_utils.util import flatten
from constants.constants import get_silicon_all_scf_data
from scf.qelattice import get_qel
from create_input.get_filtered_structure_data import get_filtered_structure_path_by_energy
from create_input.get_filtered_structure_data import get_filtered_structure_path_by_force
from descriptors.sf.get_sf_value import SfFuncVal

path2root = '/Users/y1u0d2/desktop/Lab/result/qe/Si'
# all_dirs = get_silicon_all_scf_data()
all_dirs = get_filtered_structure_path_by_force(1000)

{'mp-165': 113, 'mp-16220': 326, 'mp-149_dimer': 7, 'mp-10649': 1, 'mp-149': 574}
1025 structures were removed


In [11]:
rcut_range = list(range(4, 8, 1))
pair_range = list(range(3, 8, 1))

path2save = '/Users/y1u0d2/desktop/Lab/result/sf/Si/04'

In [12]:
from ase.io import read
import os

for rcut in rcut_range:
    param_df_list = []
    for n_pair in pair_range:
        param_df = get_sf_radial_params(
            elm=['Si'], 
            rcut=rcut, 
            nb_param_pairs=n_pair
        )
        param_df_list.append(param_df)
    param_df_concated = pd.concat(param_df_list)

    g2_params = param_df_concated[['eta', 'rs']].values

    df_list = []
    for path in all_dirs:
        try:
            qel = get_qel(path)
            force_df = pd.DataFrame(data=qel.get_force(), columns=['fx', 'fy', 'fz'])
        except:
            print(path)
            continue

        if os.path.exists(os.path.join(path, 'scf.in')):
            structure = read(filename=os.path.join(path, 'scf.in'), format='espresso-in')
        else:
            structure = read(filename=os.path.join(path, 'relax.in'), format='espresso-in')
        structure_id = qel.structure_id
        df = SfFuncVal.get_g2_value(
            g2_params=g2_params,
            structure=structure,
            species=['Si', 'Si'],
            rcut=rcut
            )
        df['structure_id'] = structure_id
        df = pd.concat([df, force_df], axis=1)
        df.drop(columns=['g1'], inplace=True)

        # df_g4 = SfFuncVal.get_g4_value(
        #     g4_params=g4_params,
        #     structure=structure,
        #     species=['Si', 'Si', 'Si'],
        #     rcut=rcut
        # )
        # df_g4.drop(columns=['g1'], inplace=True)
        
        # df = pd.concat([df, df_g4], axis=1)

        df_list.append(df)
    df_sf = pd.concat(df_list)
    df_sf.to_csv(os.path.join(path2save, f'sf_{rcut}.csv'), index=False)

# ang_param_df = get_sf_ang_params(
#     elm=['Si'],
#     rcut=rcut,
#     nb_param_pairs=3,
#     ang_type='angular_wide',
#     zetas=[1, 6],
#     r_lower=3
#     )
# g4_params = ang_param_df[['eta', 'zeta', 'lam']].values

In [ ]:
# csv 保存

# df_sf.to_csv('/Users/y1u0d2/desktop/Lab/result/sf/Si/sf_1.csv', index=False)